<a href="https://colab.research.google.com/github/yingchiehchan/Bayesian-Analysis-with-Python/blob/master/DNN_coolingload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys
sys.path.append('/content/submodel_class')

In [4]:
pip install silence_tensorflow

  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.1.1-cp36-none-any.whl size=3743 sha256=29124fc55c5cceb4cf385c324aff3875013f540a264fba635b81bdfad2166730
  Stored in directory: /root/.cache/pip/wheels/51/0b/35/cf3020764bee61daa81fa249df3a448e3806344a087fc12292
Successfully built silence-tensorflow


In [10]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
from top_factory import factory_loading

ImportError: ignored

In [ ]:
def get_loss(predy,truey):#參數不能有特殊值成員(na inf) R2會error
  dict_los={'MAE':np.nan,'RMSE':np.nan,'MAPE':np.nan,'R2':np.nan}
  dict_los['R2']=metrics.r2_score(truey,predy)
  dict_los['MAE']=metrics.mean_absolute_error(truey,predy)
  dict_los['RMSE']=metrics.mean_squared_error(truey,predy,squared=False)
  mask=truey!=0#實際為0的值我預設要剔除(會得出無限)
  dict_los['MAPE']=np.sum(np.abs((truey[mask]-predy[mask])/truey[mask]))/len(truey[mask])#MAPE sklearn只有未穩定版本
  return dict_los

def set_traintest(df,day,count,rang):
  X_trainN=df.loc[(day >= count) & (day < count+rang),:df.columns[-2]].values
  Y_trainN=df.loc[(day >= count) & (day < count+rang),df.columns[-1]].values
  X_testN=df.loc[(day == count+rang),:df.columns[-2]].values
  Y_testN=df.loc[(day == count+rang),df.columns[-1]].values
  idx=df.loc[(day == count+rang)].index
  return X_trainN,Y_trainN,X_testN,Y_testN,idx

factory=factory_loading()
load_unit=factory.get_loadObj()
load_unit.file_dict['outList']=['Secondary side kW']
load_unit.file_dict['featureList']=['estimated_Irr', 'temperature', '12_Tavg', 'humidity', 'hour', 'holiday']
#載入程序 需使用者自訂 強轉時序標籤與選擇所要Columns->trans_TimeSerDataset(path,df.index equal function,need columns)
#load_unit.trans_TimeSerDataset('/content/new_forcast_set.csv',lambda df:pd.to_datetime(df.iloc[:,1]),
#                ['estimated_Irr', 'temperature', '12_Tavg', 'humidity', 'hour','isHoliday'])
feadf=pd.read_csv('/content/2020new_forcast_setDay.csv',index_col=0)
feadf['time']=pd.to_datetime(feadf['time'])
feadf.set_index('time',inplace=True)
feadf=feadf.loc[:,['estimated_Irr', 'temperature', 'humidity','holiday']]
feadf['hour']=feadf.index.hour
feadf=feadf[['estimated_Irr', 'temperature', 'humidity', 'hour', 'holiday']]
for i in range(1,13):
  feadf[f'{i}_hour_T']=feadf['temperature'].shift(i)
load_unit.push_TimeSerData(feadf,lambda x:x.loc['2020-01-01 00:00:00':'2020-12-31 23:00:00',:])

load_unit.trans_TimeSerDataset('/content/input2020.csv', lambda df:pd.to_datetime(df.iloc[:,0]),['Secondary side kW'])
#建置dataset
load_unit.get_dataset('2019-01-01 00:00:00','2019-12-31 23:00:00','1H')
#dataset插入行 所依照的是由載入程序留下的columns決定 而不是建好的dataset
load_unit.insert_col(2,'12_Tavg',lambda x:x.loc[:,'1_hour_T':'12_hour_T'].mean(axis=1))
#縮放 並將hourmode設置
#normdf=load_unit.scaler()

modeltype='aleatory'
#modeltype='epistemic'
DNN_MODEL=factory.get_modelObj(normdf.shape[1]-1,modeltype)#選擇MODEL種類

weeknum=2
traindays=7*weeknum #周數
totaldays=367#終止天數
startingday=1#起始天數
dict_df={'idx':[],'act':[],'mean':[],'std':[],'upper':[],'lower':[]}
for i in range(startingday,totaldays-traindays):#每次滑動一天 重新complie model
  X_trainN,Y_trainN,X_testN,Y_testN,idx=set_traintest(normdf,normdf.index.dayofyear,i,traindays)
  DNN_MODEL.build_model()#重新complie model building
  if X_trainN.shape[0]>0 and X_testN.shape[0]>0:
    history =DNN_MODEL.fit(X_trainN, Y_trainN)
    print(f"{idx[-1].date()}:{history}")
    if not np.isnan(history):#模組未發散再記
        pred=DNN_MODEL.pred(X_testN)
        dict_df['act'].extend(load_unit.sclY.inverse_transform(Y_testN).ravel())
        DNN_MODEL.get_val_inverse_summary(dict_df,load_unit.sclY,pred)
        dict_df['idx'].extend(idx)

ans=pd.DataFrame(dict_df)
ans.set_index('idx',inplace=True)
ans.to_csv(f"{load_unit.file_dict['hourmode']}_{DNN_MODEL.model_dict['layer_num']}node_24HR_{DNN_MODEL.model_dict['modelType']}.csv")
pd.DataFrame(get_loss(ans['mean'],ans['act']),index=[0]).to_csv(f'loss{weeknum}w.csv')
plt.figure(figsize=(30,15))
plt.xticks(rotation=90,fontsize=25)
plt.yticks(fontsize=25)
plt.plot(ans.index,ans['act'],'r',label='act')
plt.plot(ans.index,ans['mean'],'b',label='prediction')
plt.fill_between(ans.index,ans['upper'],ans['lower'],alpha=0.3,color='g')
plt.legend(fontsize=25)
plt.savefig(f'{weeknum}w.png')

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=ans.index, y=ans['upper'],
                         mode = 'lines',
                         fillcolor='rgba(255,255,255,0)',
                         line_color='rgba(0,255,0,0.3)',
                         name="",
                         fill='tozeroy'))

fig.add_trace(go.Scatter(x=ans.index, y=ans['lower'],
                         mode = 'lines',
                         fillcolor='rgba(0,255,0,0.3)',
                         line_color='rgba(0,255,0,0.3)',
                         name="fillrange",
                         fill='tonexty'))
fig.add_trace(go.Scatter(x=ans.index, y=ans['act'],
                         mode ='lines',
                         line_color='rgba(255,0,0,1)',
                         name="act"
                         ))
fig.add_trace(go.Scatter(x=ans.index, y=ans['mean'],
                         mode ='lines',
                         line_color='rgba(0,0,255,1)',
                         name="prediction"
                         ))

fig.update_xaxes(
        tickangle = 90,
        title_font = {"size": 20},
        title_standoff = 25)

fig.update_yaxes(
        title_font = {"size": 20},
        title_standoff = 25)

fig.show()

#from google.colab import files
#files.download(f'loss{weeknum}w.csv')
#files.download(f"{load_unit.file_dict['hourmode']}_{DNN_MODEL.model_dict['layer_num']}node_24HR_{DNN_MODEL.model_dict['modelType']}.csv")
#files.download(f"{weeknum}w.png")

